# Conformal Field Theory data

This notebook contains composite types (structures) representing objects in CFTs:
* the central charge
* fields

In [1]:
#when not working on this notebook set to false to avoid running cells when importing
run_tests=true;

#packages
using BenchmarkTools

## Central charge

The central charge $c$ can be parametrized by the variables $B$, $b$ or $\beta$ such that
$$
c = 13 + 6B + 6 B^{-1} \quad , \quad B = b^2 = -\beta^2 
$$
Conversely, we have 
$$
B = \frac{c-13 \pm \sqrt{(c-1)(c-25)}}{12}
$$

We represent a central charge as a dictionary with 4 keys $b$,$\beta$,$B$,$c$, and the associated values of the parameters.

In [2]:
#=  The central charge can be given directly or as a function of b,β,B. No matter what parameter
    is passed we compute B from it, then b,β,c. 
    We ensure that the relation between b and beta never changes (sqrt branch). =#


Bfrom(parameter,value) = Dict(
        "c" => (value-13+sqrt(complex((value-1)*(value-25))))/12,
        "b" => value^2,
        "β" => -value^2,
        "B" => value
    )[parameter]

Bto(parameter,value) = Dict(
        "c" => 13+6*value+6/value,
        "b" => sqrt(complex(value)),
        "β" => im*sqrt(complex(value)),
        "B" => value
    )[parameter]

# T is the type of the parameter; either Complex{Float64} or Complex{BigFloat} for arbitrary precision.
struct CentralCharge{T}
    # T is the type of the parameterS; either Complex{Float64} or Complex{BigFloat} for arbitrary precision.
    values::Dict{String,T}

end

# Constructor
function CentralCharge(parameter="c",value=1)
    T=typeof(value)
    B=Bfrom(parameter,value)
    dict=Dict(key => Bto(key,B) for key in ("c","b","β","B"))
    CentralCharge{complex(T)}(dict)
end

#Print an object of type CentralCharge
function Base.show(io::IO,charge::CentralCharge)
    for (key,value) in charge.values
        println(io, "$key = $value")
    end
end

#overload [] to access values in charge
Base.getindex(charge::CentralCharge,key) = charge.values[key]

In [3]:
if run_tests
    #Create a charge with double precision by passing c
    charge=CentralCharge("c",0.5)
    #Create a charge with arbitrary precision
    charge2=CentralCharge("B",big(-0.75))
    #Print all parameters
    println(charge)
    #Access parameters with [""]
    println("b = ",charge2["b"])

end

B = -0.75 + 0.0im
c = 0.5 + 0.0im
b = 0.0 + 0.8660254037844386im
β = -0.8660254037844386 + 0.0im

b = 

0.0 + 0.8660254037844386467637231707529361834714026269051903140279034897259665084543988im


## Fields

Fields in 2D CFTs are elements of representations of the Virasoro algebra.

Non-logarithmic fields are elements of representations where the generators $L_0$ and $\bar L_0$ of the Virasoro algebra act diagonally, with eigenvalues called conformal weights or dimensions and respectively denoted $(\Delta,\bar \Delta)$. 
Logarithmic fields are elements of representations where $L_0$ and $\bar L_0$ act as triangular matrices; see [arXiv:2007.04190](https://arxiv.org/abs/2007.04190).

We parametrise the dimensions in terms of $P,p,\Delta,\delta$, related by

$$
\Delta = \frac{c-1}{24} + \delta  \quad , \quad \delta = -P^2 = p^2
$$

The Kac parametrisation for conformal weights is
$$ p_{(r,s)}=\frac{1}{2}(b r - b^{-1}s)$$

where $r,s$ are arbitrary numbers. We say the field is degenerate if $r,s\in \mathbb Z$ and rs>0. In terms of $r,s$, the dimension $\Delta$ is written


$$\Delta_{(r,s)} = \frac14 B (1-r^2) + \frac12 (1-rs) + \frac14\frac{1-s^2}{B}
$$

This convention is consistent with the one in in [this code](https://gitlab.com/s.g.ribault/Bootstrap_Virasoro.git)  but differs from the one in [this paper](https://arxiv.org/abs/2208.14298).

In [4]:
p_from(parameter,value,charge::CentralCharge) = Dict(
        "Δ" => sqrt(complex(value - (charge["c"]-1)/24)),
        "δ" => sqrt(complex(value)),
        "P" => -im*value,
        "p" => value
    )[parameter]

p_to(parameter,value,charge::CentralCharge) = Dict(
        "Δ" => value^2 + (charge["c"]-1)/24,
        "δ" => value^2,
        "P" => im*value,
        "p" => value
    )[parameter]

struct Field{T}

    values::Dict{String,T}
    isKac::Bool
    r::Rational
    s::Rational

end

#Constructor
function Field(parameter="",value=0,charge::CentralCharge = CentralCharge("c",1);Kac=false,r=0,s=0)
    if Kac
        p=.5*(charge["b"]*r - 1/charge["b"]*s)
        Field{complex(T)}(true, Dict(key => p_to(key,p,charge) for key in ("Δ","δ","P","p")),r,s)
    else
        p=p_from(parameter,value,charge)
        Field{complex(T)}(true, Dict(key => p_to(key,p,charge) for key in ("Δ","δ","P","p")),r,s)
    end
end

Print a field
function Base.show(io::IO,field::Field)
    for (key,value) in field.values
        println(io, "$key = $value")
    end
    if field.isKac
        println("r = ",dim.r," ; s = ",dim.s)
    end
end

In [5]:
if run_tests
    charge=CentralCharge("c",0.)
    println(charge)
#    println(Dimension(Kac=true,r=-2.0,s=-1,charge=charge))
end

B = -0.6666666666666666 + 0.0im
c = 0.0 + 0.0im
b = 0.0 + 0.816496580927726im
β = -0.816496580927726 + 0.0im



In [21]:
function test()
    p=p_from("Δ",1,CentralCharge("c",1))
    Dict(key => p_to(key,p,charge) for key in ("Δ","δ","P","p"))
end

test (generic function with 1 method)

In [22]:
test()

Dict{String, ComplexF64} with 4 entries:
  "Δ" => 0.958333+0.0im
  "P" => 0.0+1.0im
  "δ" => 1.0-0.0im
  "p" => 1.0-0.0im

In [24]:
Field{ComplexF64}(false,test(),1,1)

Field{ComplexF64}(false, Dict{String, ComplexF64}("Δ" => 0.9583333333333334 + 0.0im, "P" => 0.0 + 1.0im, "δ" => 1.0 - 0.0im, "p" => 1.0 - 0.0im), 1, 1)